In [42]:
!pip install numpy
!pip install pandas
!pip install seaborn
!pip install nltk
!pip install svgling
!pip install gensim
!pip install wordcloud

import numpy as np 
import pandas as pd
from IPython.display import display
import seaborn as sns
import re
import string
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, ne_chunk # for NER
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/test/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/test/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/test/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /Users/test/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/test/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/test/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# 1-Data Exploration

###### Before preprocessing the dataset, we will explore, prepare, analyze and visualize the datase

In [21]:
# Load the Dataset

colnames = ['sentiment', 'title', 'text']

train_df = pd.read_csv('archive/train.csv', names=colnames)
test_df = pd.read_csv('archive/test.csv' , names=colnames)


In [22]:
# the dataset has been successfully loaded 
print(display(train_df.head()))
print(display(test_df.head()))

,sentiment,title,text
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


None


,sentiment,title,text
0,2,Great CD,My lovely Pat has one of the GREAT voices of h...
1,2,One of the best game music soundtracks - for a...,Despite the fact that I have only played a sma...
2,1,Batteries died within a year ...,I bought this charger in Jul 2003 and it worke...
3,2,"works fine, but Maha Energy is better",Check out Maha Energy's website. Their Powerex...
4,2,Great for the non-audiophile,Reviewed quite a bit of the combo players and ...


None


###### The two dataframes have three columns:

sentiment - Target variable with two categorical levels: 1 if the review is negative (1/2 stars rating); 2 if the review is positive (4/5 stars rating).

title - Heading of the review.

text - Body of the review.

In [23]:
# Inspect the structure of the dataset 

print(train_df.shape)
print(test_df.shape)
print ("\n\n")
print(display(train_df.info()))
print ("\n\n")
print(test_df.info())



(3600000, 3)
(400000, 3)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600000 entries, 0 to 3599999
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   sentiment  int64 
 1   title      object
 2   text       object
dtypes: int64(1), object(2)
memory usage: 82.4+ MB


None

None



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentiment  400000 non-null  int64 
 1   title      399976 non-null  object
 2   text       400000 non-null  object
dtypes: int64(1), object(2)
memory usage: 9.2+ MB
None


Since the original dataset is huge, for time and memory contsraints we will restrict to a random subset of 50000 rows from train_df and 10000 rows from test_df, which will be respectively our training and validation set. We select such subsets randomly and so that their are both perfectly balanced. We also merge the title and text features in a single text column.

In [24]:
# fill empty titles with empty string
train_df['title'].fillna('', inplace=True)
test_df['title'].fillna('', inplace=True)

# take sample of data for this project
train_len = 50000
test_len = 10000
rs = 42

df = pd.concat([train_df.loc[train_df['sentiment'] == 1].sample(train_len//2, random_state=rs),
                train_df.loc[train_df['sentiment'] == 2].sample(train_len//2, random_state=rs),
                test_df.loc[test_df['sentiment'] == 1].sample(test_len//2, random_state=rs),
                test_df.loc[test_df['sentiment'] == 2].sample(test_len//2, random_state=rs)]).reset_index(drop=True)

# combine title and text
df['text'] = df['title'] + '. ' + df['text']
df.drop('title', axis=1, inplace=True)
df.head() 



,sentiment,text
0,1,The History of Torture and Execution. I have n...
1,1,Stainless Steel shouldn't corrode... should it...
2,1,Another Weak Sequel To A Once Promising Franch...
3,1,"Simply Awful. Contrived, trite, ""Bridget Jones..."
4,1,"FLASH. Jeff Beck's Flash In my opinion, was no..."


In [25]:
print(f'Label counts - training set:\n{df[:train_len].sentiment.value_counts()}')
print(f'\nLabel counts - validation set:\n{df[train_len:].sentiment.value_counts()}')
# checking for missing values
df.isnull().sum() 

Label counts - training set:
sentiment
1    25000
2    25000
Name: count, dtype: int64

Label counts - validation set:
sentiment
1    5000
2    5000
Name: count, dtype: int64


sentiment    0
text         0
dtype: int64

In [26]:
# plot the classes
sns.countplot(x=df["sentiment"]); 

# 2-Text Preprocessing

In [8]:
# function to process text, including Tokenizing, Lemmatizing and Stemmatizinh, Stop Words removals and NER
def preprocess_text(df, text_column, processed_column='text_processed', tokens_column='text_tokens', lemma_column='text_lemmatize', entities_column='text_entities'):
    # Convert text to lowercase and remove punctuation, numbers, and line breaks
    df[processed_column] = df[text_column].str.lower().apply(lambda x: re.sub(r'[^\w\s]|\d|\r|\n', '', x))
    
    # Tokenize the text of processed text
    df[tokens_column] = df[processed_column].apply(lambda x: nltk.word_tokenize(x))
    
    # Remove stop words from tokenized text
    stop_words = set(stopwords.words('english'))
    df[tokens_column] = df[tokens_column].apply(lambda x: [word for word in x if word not in stop_words])
    
    # Remmove suffixes with Lemma and Stemma and find roots
    df[lemma_column] = df[tokens_column].apply(lambda x: [stemmer.stem(lemmatizer.lemmatize(word)) for word in x])
    
    # NER - Extract named entities
    df[entities_column] = df[tokens_column].apply(lambda x: ne_chunk(pos_tag(x)))
    
    
    # Join the tokens back into a single string
#     df['text'] = df['text'].apply(' '.join)
    
    
    return df

In [27]:
df = preprocess_text(df, 'text')

In [37]:
# Let's have a look at our processed Data Frame
print(df.shape)
df.head()

(60000, 7)


,sentiment,text,text_processed,text_tokens,text_lemmatize,text_entities,topics
0,1,The History of Torture and Execution. I have n...,the history of torture and execution i have no...,"[history, torture, execution, yet, received, b...","[histori, tortur, execut, yet, receiv, book, o...","[(history, NN), (torture, NN), (execution, NN)...","[(5, 0.51496077), (7, 0.45424443)]"
1,1,Stainless Steel shouldn't corrode... should it...,stainless steel shouldnt corrode should it we ...,"[stainless, steel, shouldnt, corrode, really, ...","[stainless, steel, shouldnt, corrod, realli, p...","[(stainless, NN), (steel, NN), (shouldnt, NN),...","[(4, 0.023144903), (5, 0.96175444)]"
2,1,Another Weak Sequel To A Once Promising Franch...,another weak sequel to a once promising franch...,"[another, weak, sequel, promising, franchise, ...","[anoth, weak, sequel, promis, franchis, hellra...","[(another, DT), (weak, JJ), (sequel, NN), (pro...","[(3, 0.38876757), (4, 0.23142038), (7, 0.35280..."
3,1,"Simply Awful. Contrived, trite, ""Bridget Jones...",simply awful contrived trite bridget joness di...,"[simply, awful, contrived, trite, bridget, jon...","[simpli, aw, contriv, trite, bridget, jone, di...","[(simply, RB), (awful, JJ), (contrived, VBN), ...","[(2, 0.04256444), (7, 0.73173743), (8, 0.19872..."
4,1,"FLASH. Jeff Beck's Flash In my opinion, was no...",flash jeff becks flash in my opinion was not o...,"[flash, jeff, becks, flash, opinion, one, bett...","[flash, jeff, beck, flash, opinion, one, bette...","[(flash, NN), (jeff, NN), (becks, NNS), (flash...","[(0, 0.035139356), (4, 0.21914667), (5, 0.0979..."


In [ ]:
from gensim.corpora import Dictionary
from gensim.models import LdaModel

def topic_modeling(documents, dictionary, num_topics=10, num_words=10):
    # Create a corpus from the documents
    corpus = [dictionary.doc2bow(doc) for doc in documents]
    
    # Train the LDA model on the corpus
    lda = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
    
    # Print the top words for each topic
    for i, topic in lda.show_topics(num_words=num_words, formatted=False):
        print(f'Topic {i}: {[word for word, _ in topic]}')
    
    return lda

# Extract the list of documents from the DataFrame
documents = df['text_tokens'].tolist()

# Create a dictionary of the documents
dictionary = Dictionary(documents)

# Apply the topic modeling function to the documents
lda = topic_modeling(documents, dictionary=dictionary)

# Assign the topic assignments to a new column in the DataFrame
df['topics'] = [lda[dictionary.doc2bow(doc)] for doc in documents]

In [ ]:
from wordcloud import WordCloud

def visualize_topic(lda, dictionary, topic_id):
    # Create a list of the top words for the topic
    top_words = [word for word, _ in lda.show_topic(topic_id, topn=10)]
    
    # Create a word cloud for the topic
    wordcloud = WordCloud(width=600, height=400).generate(' '.join(top_words))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f'Topic {topic_id}')
    plt.show()

# Extract the list of documents from the DataFrame
documents = df['text_tokens'].tolist()

# Create a dictionary of the documents
dictionary = Dictionary(documents)

# Train the LDA model on the corpus
lda = LdaModel([dictionary.doc2bow(doc) for doc in documents], id2word=dictionary)

# Visualize the topic with ID 3
visualize_topic(lda, dictionary, 3)

# **3. Vectorization**

Feature Extraction: Transform the preprocessed text into numerical representations that machine learning models can understand. Popular methods include bag-of-words, TF-IDF (Term Frequency-Inverse Document Frequency), or word embeddings like Word2Vec or GloVe. Experiment with different approaches and choose the one that suits your dataset and analysis goals.


## **3.1 Tf-Idf Vectorization (sparse)**

In [45]:
df['text_tokens']

0        [history, torture, execution, yet, received, b...
1        [stainless, steel, shouldnt, corrode, really, ...
2        [another, weak, sequel, promising, franchise, ...
3        [simply, awful, contrived, trite, bridget, jon...
4        [flash, jeff, becks, flash, opinion, one, bett...
                               ...                        
59995    [good, value, cable, connected, p, lcd, tvit, ...
59996    [waynes, war, john, waynes, answer, antiwar, m...
59997    [great, history, lesson, must, anyone, interes...
59998    [super, quiet, super, quiet, efficient, upgrad...
59999    [great, book, great, book, found, acutally, pr...
Name: text_tokens, Length: 60000, dtype: object

In [46]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

# Tokenize content by whitespace
tokenized_content = df['text_tokens']
dct = Dictionary(tokenized_content)  # fit dictionary
corpus = [dct.doc2bow(line) for line in tokenized_content]
tfidf_model = TfidfModel(corpus)

print('Number of features: {}'.format(len(dct)))

Number of features: 129720


In [47]:
doc = 0 #Check for our first review in our dataset
print('Text 0:\n{}\n'.format(df['text_processed'][0]))

# Inspect tf-idf scores of document sorted by the similarity score
for w,s in sorted(tfidf_model[corpus[doc]], reverse=True, key=lambda x: x[1]):
    print('{}:{}'.format(dct[w], s))

Text 0:
the history of torture and execution i have not yet received the book ordered even though i paid a rather high price for it i have been unable to get a response from the seller and am beginning to think that they may be something less than reputablejames j ebyrating zero stars

ebyrating:0.4193329258622823
reputablejames:0.4193329258622823
execution:0.2614219059538113
torture:0.25000616307534146
j:0.24657788503747421
unable:0.21173652595411027
response:0.21060354531175754
zero:0.19661266217016907
seller:0.18418500344497338
paid:0.1714862406313785
beginning:0.16768255556428044
history:0.1510574384987113
rather:0.14658312985119648
ordered:0.14284792163721538
received:0.14103200412476954
high:0.13733633053217176
less:0.13593938963572713
yet:0.13341638525858823
stars:0.1322760965329755
may:0.1271254673152254
though:0.12085458690345048
price:0.11475507799014664
something:0.11188633776083111
think:0.09742628647411378
even:0.0809706506759444
get:0.06978335515098247
book:0.052657103561

In [50]:
from gensim import matutils
# vectorize all documents with tf-idf
tfidf_vectorization_csr = matutils.corpus2csc(tfidf_model[corpus], num_terms=len(dct))
X_tfidf_vectorization = tfidf_vectorization_csr.T.toarray()

Topic 0: ['water', 'coffee', 'hot', 'cup', 'maker', 'smell', 'pot', 'terrific', 'photo', 'cake']
Topic 1: ['comfortable', 'seat', 'shoes', 'helps', 'tape', 'headphones', 'pair', 'wear', 'satisfied', 'terms']
Topic 2: ['cats', 'musicians', 'frame', 'sports', 'gun', 'politics', 'performed', 'tube', 'pans', 'inspired']
Topic 3: ['great', 'game', 'one', 'like', 'get', 'little', 'old', 'love', 'good', 'bought']
Topic 4: ['movie', 'dvd', 'watch', 'film', 'movies', 'video', 'season', 'seen', 'show', 'see']
Topic 5: ['product', 'great', 'use', 'good', 'one', 'would', 'works', 'get', 'work', 'price']
Topic 6: ['blues', 'town', 'complaints', 'intended', 'remarkable', 'aspects', 'lp', 'xbox', 'drums', 'billy']
Topic 7: ['book', 'read', 'one', 'good', 'great', 'story', 'like', 'books', 'would', 'really']
Topic 8: ['cd', 'album', 'music', 'like', 'great', 'one', 'songs', 'best', 'good', 'love']
Topic 9: ['dog', 'de', 'la', 'account', 'dogs', 'que', 'en', 'el', 'doll', 'spanish']


In [38]:
df.topics[3]

[(2, 0.04256444), (7, 0.73173743), (8, 0.19872497)]

